# Geodetic motion in Schwarzschild geometry

### The metric and the geodesic equation

The metric of the Schwarzschild space around a not charged, not spinnig black hole is $$ ds^2 = -(1-\frac{2GM}{r})c^2 dt^2 +(1-\frac{2GM}{r})^{-1} dr^2 + r^2 d\theta^2 + r^2 \sin(\theta)^2 d\phi^2 $$ 
The metric is described by $ g_{\mu \nu} = diag(-(1-\frac{2GM}{r}), (1-\frac{2GM}{r})^{-1}, r^2, r^2\sin(\theta)^2) $ 
 
The evolution of a body in geodesic motion is described by
$$ \begin{equation}
    \begin{cases}
      \frac{dx^\mu}{d \tau}= u^\mu\\
      \frac{du^\alpha}{d \tau} + \Gamma^\alpha_{\beta \gamma}u^\beta u^\gamma = 0
    \end{cases}\
\end{equation} $$
The equation can be integrated in the proper time to obtain the coordinate time and the position of the body $ x^\mu = (t, r, \theta, \phi) $ 
 
  
The non-zero Christoffel symbols are $ {\Gamma^\alpha_{\beta \gamma}} $

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as ani
import pandas as pd
from scipy.integrate import solve_ivp

### Restricted equations

In this simulations I'm using $ c=1 $ and $ r = GM $, so the metric becomes $$ ds^2 = -(1-\frac{2}{r})dt^2 +(1-\frac{2}{r})^{-1} dr^2 + (GMr)^2 d\theta^2 + (GMr)^2 \sin(\theta)^2 d\phi^2 $$ 

In [10]:
G = 6.6743e-11
M = 1.98847e30
c = 299792458

R_s = (G*M)/c**2  #Schwarzschild radius in meters

print(f'The Schwarzschild radius of the system is {R_s:.4e} m')

The Schwarzschild radius of the system is 1.4767e+03 m
